# Sparse Hebbian Learning: basics

We are interested here in learning the "optimal" components of a set of images (let's say some "natural", usual images). As there is no supervisor to guide the learning, this is called unsupervised learning. Our basic hypothesis to find the best ("optimal") components will be to assume that *a priori* the most sparse is more plausible. We will implement the derived algorithm in this set of scripts.

Here, we will show the basic operations that are implemented in this package. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4, suppress=True)

## experiments

To test and control for the role of different parameters, we will have a first object (in the [shl_experiments.py](https://github.com/bicv/SHL_scripts/blob/master/shl_scripts/shl_experiments.py) script) that controls a learning experiment. It contains all relevant parameters, but can also keep a trace of the history of some statistics. This is useful to compare the relative efficiency of the different solutions.


In [3]:
matname = 'basics'

In [4]:
from shl_scripts.shl_experiments import SHL

In [5]:
shl = SHL(verbose=10)
help(shl)

Help on SHL in module shl_scripts.shl_experiments object:

class SHL(builtins.object)
 |  Base class to define SHL experiments:
 |      - initialization
 |      - coding and learning
 |      - visualization
 |      - quantitative analysis
 |  
 |  Methods defined here:
 |  
 |  __init__(self, height=256, width=256, patch_width=16, N_patches=65536, datapath='../database/', name_database='kodakdb', do_mask=False, do_bandpass=True, over_patches=16, patch_ds=1, n_dictionary=441, learning_algorithm='mp', fit_tol=None, l0_sparseness=14, alpha_MP=0.9, one_over_F=True, n_iter=2049, eta=0.02, beta1=0.9, beta2=0.999, epsilon=1e-08, do_precision=False, eta_precision=0.0005, homeo_method='HEH', eta_homeo=0.02, alpha_homeo=0.5, C=3.0, nb_quant=128, P_cum=None, do_sym=False, seed=42, patch_norm=False, batch_size=32, record_each=128, record_num_batches=1024, n_image=None, DEBUG_DOWNSCALE=1, verbose=0, cache_dir='cache_dir')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 

In [6]:
!ls -l {shl.cache_dir}/{matname}*

-rw-r--r--  1 lolo  staff  231154688 Sep 26 11:20 cache_dir/basics_coding.npy
-rw-r--r--  1 lolo  staff  102735488 Sep 26 10:55 cache_dir/basics_data.npy
-rw-r--r--  1 lolo  staff    2540822 Sep 26 11:19 cache_dir/basics_dico.pkl


In [7]:
!rm {shl.cache_dir}/{matname}*
!ls -l {shl.cache_dir}/{matname}*

ls: cache_dir/basics*: No such file or directory


In [8]:
data = shl.get_data(matname=matname)
print('number of patches, size of patches = ', data.shape)
print('average of patches = ', data.mean(), ' +/- ', data.mean(axis=1).std())
SE = np.sqrt(np.sum(data**2, axis=1))
print('average energy of data = ', SE.mean(), '+/-', SE.std())

Extracting data..No cache found cache_dir/basics_data: Extracting data... Extracting data..bird08.png, yellowleaves39.png, rocky10.png, bittern62.png, flowerhill29.png, clouds43.png, goldwater67.png, calcoast09.png, yose05.png, woods54.png, koala52.png, bora04.png, flowers37.png, yose07.png, cattails70.png, hibiscus30.png, cucorn50.png, geyser27.png, reflection63.png, craterlake12.png, Data is of shape : (65520, 256) - done in 14.26s.
Data is of shape : (65520, 256) - done in 14.73s.
number of patches, size of patches =  (65520, 256)
average of patches =  -4.193836974888984e-19  +/-  0.01106396040570928
average energy of data =  4.846976510135289 +/- 1.457199891149408


## learning

The actual learning is done in a second object (here ``dico``) from which we can access another set of properties and functions  (see the [shl_learn.py](https://github.com/bicv/SHL_scripts/blob/master/shl_scripts/shl_learn.py) script):

In [ ]:
list_figures = ['show_dico', 'time_plot_error', 'time_plot_logL', 'show_Pcum']#, 'plot_variance',  'plot_variance_histogram',  'time_plot_prob',  'time_plot_kurt',  'time_plot_var']
dico = shl.learn_dico(data=data, list_figures=list_figures, matname=matname)

No cache found cache_dir/basics_dico.pkl: Learning the dictionary with algo = mp 
 Training on 65520 patches
Iteration   1 /   2049 (elapsed time:   0s,   0mn   0s)
Iteration  129 /   2049 (elapsed time:  18s,   0mn  18s)
Iteration  257 /   2049 (elapsed time:  37s,   0mn  37s)
Iteration  385 /   2049 (elapsed time:  55s,   0mn  55s)
Iteration  513 /   2049 (elapsed time:  72s,   1mn  12s)
Iteration  641 /   2049 (elapsed time:  91s,   1mn  31s)
Iteration  769 /   2049 (elapsed time:  109s,   1mn  49s)
Iteration  897 /   2049 (elapsed time:  128s,   2mn   8s)
Iteration  1025 /   2049 (elapsed time:  146s,   2mn  26s)
Iteration  1153 /   2049 (elapsed time:  164s,   2mn  44s)
Iteration  1281 /   2049 (elapsed time:  182s,   3mn   2s)
Iteration  1409 /   2049 (elapsed time:  201s,   3mn  21s)
Iteration  1537 /   2049 (elapsed time:  219s,   3mn  39s)
Iteration  1665 /   2049 (elapsed time:  238s,   3mn  58s)


In [ ]:
help(dico)

In [ ]:
print('size of dictionary = (number of filters, size of imagelets) = ', dico.dictionary.shape)
print('average of filters = ',  dico.dictionary.mean(axis=1).mean(), 
      '+/-',  dico.dictionary.mean(axis=1).std())
SE = np.sqrt(np.sum(dico.dictionary**2, axis=1))
print('average energy of filters = ', SE.mean(), '+/-', SE.std())

## coding

The learning itself is done via a gradient descent but is highly dependent on the coding / decoding algorithm. This belongs to a another function (in the [shl_encode.py](https://github.com/bicv/SHL_scripts/blob/master/shl_scripts/shl_encode.py) script)

In [ ]:
sparse_code = shl.code(data, dico, matname=matname, l0_sparseness=45)
print('number of codes, size of codewords = ', sparse_code.shape)
print('average of codewords = ', sparse_code.mean())
print('average energy of codewords = ', sparse_code.std(axis=0).mean())
print('std of the average of individual patches = ', sparse_code.mean(axis=0).std())

In [ ]:
patches = sparse_code @ dico.dictionary
print('number of codes, size of reconstructed images = ', patches.shape)

In [ ]:
error = data - patches
print('average of residual patches = ', error.mean(), '+/-', error.mean(axis=1).std())
SE = np.sqrt(np.sum(error**2, axis=1))
print('average energy of residual = ', SE.mean(), '+/-', SE.std())

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts